# Air Quality Sensors
## Around the World
PJB 2019-11-06

Purpose: a quick mapping exercise to see where air quality sensors are located.
Data downloaded from:
- https://aqs.epa.gov/aqsweb/airdata/download_files.html
- https://www.purpleair.com/sensorlist

### Look at EPA stations first

In [1]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import folium

import cmocean

In [2]:
filename = 'annual_conc_by_monitor_2019.csv'
filepath = os.path.join('data', filename)
df_epa = pd.read_csv(filepath)
df_epa.describe()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (38,40,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Year,Observation Count,Observation Percent,...,4th Max Value,1st Max Non Overlapping Value,2nd Max Non Overlapping Value,99th Percentile,98th Percentile,95th Percentile,90th Percentile,75th Percentile,50th Percentile,10th Percentile
count,16391.000000,16391.000000,16391.000000,16391.000000,16391.000000,16391.000000,16391.000000,16391.0,16391.000000,16391.000000,...,16193.000000,152.000000,152.000000,16391.000000,16391.000000,16391.000000,16391.000000,16391.000000,16391.000000,16391.000000
mean,27.099506,69.526936,724.876274,63687.669880,2.762980,39.230008,-95.368306,2019.0,1041.578915,20.562443,...,86.139922,0.978289,0.859211,85.883184,83.846515,80.803748,77.963992,73.026207,67.343792,58.682152
std,16.827142,81.295528,1763.760883,19205.835263,9.377449,3.936028,16.214881,0.0,2329.747239,12.345024,...,237.583915,0.476871,0.389150,234.838381,234.032483,232.752587,230.858451,228.225501,225.553741,222.461576
min,1.000000,1.000000,1.000000,11101.000000,1.000000,19.430800,-155.257800,2019.0,1.000000,0.000000,...,-14.000000,0.200000,0.100000,-7.300000,-7.300000,-7.300000,-7.300000,-7.300000,-12.000000,-27.000000
25%,8.000000,21.000000,6.000000,44201.000000,1.000000,36.755885,-110.829670,2019.0,27.000000,13.000000,...,0.800000,0.600000,0.600000,0.900000,0.800000,0.600000,0.500000,0.223500,0.100000,0.037000
50%,26.000000,51.000000,20.000000,62101.000000,1.000000,39.541515,-90.411095,2019.0,515.000000,23.000000,...,12.400000,0.950000,0.800000,16.300000,15.100000,12.900000,11.000000,7.900000,5.000000,1.800000
75%,39.000000,95.000000,651.000000,88101.000000,2.000000,41.603159,-82.458860,2019.0,1995.000000,25.000000,...,34.000000,1.300000,1.100000,34.000000,30.600000,25.300000,22.700000,15.900000,10.700000,5.000000
max,80.000000,840.000000,9997.000000,88503.000000,99.000000,63.723200,-67.060728,2019.0,34018.000000,100.000000,...,3860.000000,3.200000,2.300000,3550.000000,3550.000000,3550.000000,3360.000000,2950.000000,2780.000000,1970.000000


In [3]:
# How many unique sites are there?
len(df_epa.loc[:, 'Site Num'].unique())

294

### We'll lose non-numeric data but this will give us lat/lon for unique `Site Num`

In [4]:
df_epa_stations = df_epa.groupby(by = 'Latitude').mean()
df_epa_stations.reset_index(inplace = True)
df_epa_stations.head()

,Latitude,State Code,County Code,Site Num,Parameter Code,POC,Longitude,Year,Observation Count,Observation Percent,...,4th Max Value,1st Max Non Overlapping Value,2nd Max Non Overlapping Value,99th Percentile,98th Percentile,95th Percentile,90th Percentile,75th Percentile,50th Percentile,10th Percentile
0,19.430800,15.0,1.0,5.0,53680.357143,1.285714,-155.257800,2019.0,5208.071429,24.714286,...,67.613571,NaN,NaN,54.775714,50.992143,45.477857,35.734286,21.777143,15.378571,11.928571
1,25.391220,12.0,86.0,30.0,62139.875000,1.000000,-80.680819,2019.0,2127.125000,24.625000,...,80.052500,NaN,NaN,76.197500,73.718750,70.151250,66.157500,59.673750,48.351250,28.525000
2,27.849200,12.0,61.0,9991.0,44201.000000,1.000000,-80.455400,2019.0,1245.000000,15.250000,...,0.049500,NaN,NaN,0.056750,0.054500,0.051250,0.046750,0.045000,0.041250,0.032000
3,29.302650,48.0,43.0,101.0,56160.250000,1.000000,-103.177810,2019.0,1969.750000,22.833333,...,54.913250,NaN,NaN,52.856000,51.916917,49.523167,46.362250,40.357167,29.630250,12.610333
4,29.679301,22.0,109.0,1.0,68106.500000,1.000000,-90.779731,2019.0,2.000000,2.000000,...,NaN,NaN,NaN,389.100000,389.100000,389.100000,389.100000,389.100000,389.100000,388.350000


### Make a Folium plot
- Comment out all but one of the `map_1 = [...]` chunks

In [22]:
# Or, using tileset from https://leaflet-extras.github.io/leaflet-providers/preview/

# map_1 = folium.Map(location = [32.7, -117.6],
#                    zoom_start = 10)
# 
mapbox_access_token = os.environ['MY_MAPBOX_KEY']
map_1 = folium.Map(location = [40,-97.5],
                   zoom_start = 5,
                   tiles = 'CartoDB dark_matter')


# map_1 = folium.Map(location = [40,-97.5],
#                    zoom_start = 5,
#                    tiles='https://api.mapbox.com/v4/mapbox.run-bike-hike/{z}/{x}/{y}.png?access_token='+mapbox_access_token,
#                    attr='© <a href="https://www.mapbox.com/about/maps/">Mapbox</a>')

# I can add marker one by one on the map
for i in range(0, len(df_epa_stations)):
#     folium.Marker([df_epa_stations.iloc[i]['Latitude'], 
#                    df_epa_stations.iloc[i]['Longitude']], 
#                   popup=df_epa_stations.iloc[i]['Site Num']).add_to(map_1)
    folium.Circle(
        radius=100,
        location=[df_epa_stations.iloc[i]['Latitude'], 
                   df_epa_stations.iloc[i]['Longitude']],
        popup=None,
        color='green',
        fill=False,
    ).add_to(map_1)

map_1

map_1.save(outfile = 'AQ_Sensor_Map.html', closefile = 'True')

In [6]:
df_epa_stations.Longitude

0      -155.257800
1       -80.680819
2       -80.455400
3      -103.177810
4       -90.779731
           ...    
1402   -102.401800
1403   -122.456463
1404   -122.688888
1405   -122.704700
1406   -148.967600
Name: Longitude, Length: 1407, dtype: float64